# Attack Password VI (Hashes)

In [ ]:
%run '../util/Metadata.ipynb'
print_metadata()

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Recap-side-channel-attacks" data-toc-modified-id="Recap-side-channel-attacks-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Recap side channel attacks</a></span></li><li><span><a href="#Comparison-of-passwords-hashes" data-toc-modified-id="Comparison-of-passwords-hashes-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Comparison of <del>passwords</del> hashes</a></span><ul class="toc-item"><li><span><a href="#Advantages-of-hash-values-when-storing-passwords" data-toc-modified-id="Advantages-of-hash-values-when-storing-passwords-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Advantages of hash values when storing passwords</a></span></li></ul></li><li><span><a href="#Basic-Setup" data-toc-modified-id="Basic-Setup-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Basic Setup</a></span></li><li><span><a href="#Helper-Functions-for-Password-Attack" data-toc-modified-id="Helper-Functions-for-Password-Attack-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Helper Functions for Password Attack</a></span></li><li><span><a href="#Check-password-hash" data-toc-modified-id="Check-password-hash-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Check password hash</a></span></li><li><span><a href="#Disconnect" data-toc-modified-id="Disconnect-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Disconnect</a></span></li></ul></div>

In this example we want to present the final "solution" against a side-channel attack on our password check for now.

## Recap side channel attacks

We have learned how to hide critical operations but it is just a matter of effort to beat against these countermeasures.

An obvious solution against all side channel attacks would (of course) be not to give any relationship between the password and the attempt. Or more generally, between a secret and an input. This sounds weird as the purpose of a password-check-routine is to compare a password and an attempt. But one can compare not the password itself but a kind of substitute which is computed out of the password but from which the password cannot be revealed. This is exactly what a hash function accomplishes.

## Comparison of ~~passwords~~ hashes

Let's consider the following password check:

```c
uint8_t check_password_sha(uint8_t *attempt)
{
    uint8_t passbad;
    uint8_t buffer[HASH_LENGTH_SHA256];

    sha256(buffer, attempt, sizeof(password);

    trigger_high();
    passbad = memcmp(buffer, password_sha256);
    trigger_low();

    return passbad == 1;
}
```

We replaced all the elaborated side channel countermeasures by the very first approach using `memcmp`. But, as stated above, the password itself does not go into the comparison anymore.

### Advantages of hash values when storing passwords

* If an attacker reveals the input of the `memcmp` he has no more information as there is no way to reverse-compute the hash function.
* The length of the hashed password independent of the length of the password.

Besides the side channel topic there are further advantages of hash function when dealing with passwords:

* If the password is stored in plaintext somewhere the threat is high that it can be readout by an mistake in software or by another attack.

## Basic Setup

Define Variables

In [ ]:
%run "../util/Init.ipynb"

Build target and upload

**NOTE**: In my CW version ([e2adf19](https://github.com/newaetech/chipwhisperer/commit/e2adf1938804413828cc8fa8b19e3cd741950756)) the compilation of mbedtls does not work as expected. To get it work apply the following patch in the file `hardware/victims/firmware/crypto/mbedtls/include/mbedtls/config.h`:

```diff
@@ -24,7 +24,7 @@
  *
  *  This file is part of mbed TLS (https://tls.mbed.org)
  */
-#if PLATFORM==CWNANO
+#if 0
 #ifndef MBEDTLS_CONFIG_H
 #define MBEDTLS_CONFIG_H
``` 


In [ ]:
TARGET = 'sha-passwordcheck'
CRYPTO_TARGET = "MBEDTLS"
CRYPTO_OPTIONS = "SHA1"
%store TARGET
%run "$HELPERSCRIPTS/Prepare.ipynb"

Import helper functions

In [ ]:
%run "$HELPERSCRIPTS/Setup_Generic.ipynb"

In [ ]:
scope.adc.samples = 2000

## Helper Functions for Password Attack

In [ ]:
from bokeh.plotting import figure, show 
from bokeh.io import output_notebook
from bokeh.models import CrosshairTool, Label

output_notebook()

In [ ]:
import warnings
import random
import tqdm
import numpy as np

command = '4'
"""Command to execute"""

password_length = 8
"""Number of bytes of password"""

sha1_digest_length = 20
"""Number of bytes of a SHA1 digest"""

def capture(command, data):
    scope.arm()
    target.simpleserial_write(command, data)

    ret = scope.capture()

    i = 0
    while not target.is_done():
        i += 1
        time.sleep(0.05)
        if i > 100:
            warnings.warn("Target did not finish operation")
            return None

    if ret:
        warnings.warn("Timeout happened during capture")
        return None

    return scope.get_last_trace()

def target_set_password(password):
    target.simpleserial_write('p', password)
    return target.simpleserial_read('r', sha1_digest_length)

def target_check_password(command, password):
    target.simpleserial_write(command, password)
    return bytes(target.simpleserial_read('r', 1))[0] == 0

## Check password hash

In [ ]:
import hashlib
password = b'ifx2019a'
password_hash = bytes(target_set_password(password))
password_hash == hashlib.sha1(password).digest()

## Disconnect

In [ ]:
scope.dis()
target.dis()